In [117]:
!pip install nltk scikit-learn pandas numpy -q

In [118]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [119]:
movies=pd.read_csv('/content/netflix_dataset.csv')

In [120]:
movies.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [121]:
movies.tail()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
8802,s8803,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,"November 20, 2019",2007,R,158 min,"Cult Movies, Dramas, Thrillers","A political cartoonist, a crime reporter and a..."
8803,s8804,TV Show,Zombie Dumb,NaN,NaN,NaN,"July 1, 2019",2018,TV-Y7,2 Seasons,"Kids' TV, Korean TV Shows, TV Comedies","While living alone in a spooky town, a young g..."
8804,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,"November 1, 2019",2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...
8805,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,"January 11, 2020",2006,PG,88 min,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero..."
8806,s8807,Movie,Zubaan,Mozez Singh,"Vicky Kaushal, Sarah-Jane Dias, Raaghav Chanan...",India,"March 2, 2019",2015,TV-14,111 min,"Dramas, International Movies, Music & Musicals",A scrappy but poor boy worms his way into a ty...


In [122]:
movies.isnull().sum()

,0
show_id,0
type,0
title,0
director,2634
cast,825
country,831
date_added,10
release_year,0
rating,4
duration,3


In [123]:
movies['director'] = movies['director'].fillna('')

movies['cast'] = movies['cast'].fillna('')

In [124]:
movies.isnull().sum()

,0
show_id,0
type,0
title,0
director,0
cast,0
country,831
date_added,10
release_year,0
rating,4
duration,3


No more NaN values in important features columns

In [125]:
movies.insert(0,'index',movies.index)

In [126]:
movies.head()

,index,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,1,s2,TV Show,Blood & Water,,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,3,s4,TV Show,Jailbirds New Orleans,,,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,4,s5,TV Show,Kota Factory,,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [127]:
movies['combined_features'] = movies['director'] + ' '+movies['cast'] + ' '+movies['listed_in']+ ' '+movies['description']

In [128]:
data = movies[['index', 'title', 'combined_features']]

Refining the combined_features column

In [129]:
stop_words = set(stopwords.words('english'))

In [130]:
def refined_text(text):
  text = re.sub(r"[^a-zA-Z\s]", "", text) #removing special chars & nums
  text = text.lower()
  #tokenizing i.e., dividing into separate words
  tokens = word_tokenize(text)
  tokens = [word for word in tokens if word.lower() not in stop_words] #removing stop_words
  return " ".join(tokens)

In [131]:
data['redefined_text'] = data['combined_features'].apply(refined_text)

/tmp/ipython-input-1606630211.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['redefined_text'] = data['combined_features'].apply(refined_text)


In [132]:
data.head()

,index,title,combined_features,redefined_text
0,0,Dick Johnson Is Dead,Kirsten Johnson Documentaries As her father n...,kirsten johnson documentaries father nears end...
1,1,Blood & Water,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaba...",ama qamata khosi ngema gail mabalane thabang m...
2,2,Ganglands,"Julien Leclercq Sami Bouajila, Tracy Gotoas, S...",julien leclercq sami bouajila tracy gotoas sam...
3,3,Jailbirds New Orleans,"Docuseries, Reality TV Feuds, flirtations an...",docuseries reality tv feuds flirtations toilet...
4,4,Kota Factory,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam ...",mayur jitendra kumar ranjan raj alam khan ahsa...


Converting from textual to numerical data

In [133]:
tfidf=TfidfVectorizer()

feature_vectors=tfidf.fit_transform(data['redefined_text'])

In [134]:
similarity=cosine_similarity(feature_vectors)

In [135]:
similarity.shape

(8807, 8807)

In [136]:
#recommendation function
def recommend_movies(movie_name, similarity=similarity, df=data, top_n=10):
  list_of_titles=df['title'].tolist()
  close_match = difflib.get_close_matches(movie_name, list_of_titles)
  closest_match=close_match[0]
  index_of_movie = df[df.title==closest_match]['index'].values[0]

  #get similarity scores
  similarity_score = list(enumerate(similarity[index_of_movie]))
  sorted_similar_movies =sorted(similarity_score, key=lambda x:x[1], reverse=True)
  similarity_score = sorted_similar_movies[1:top_n+1]

  movie_indices = [i[0] for i in similarity_score]

  return df['title'].iloc[movie_indices]

Testing the working of the model

In [137]:
movie_name = data['title'][11]
print(movie_name)

Bangkok Breaking


In [138]:
print(f"Recommendation for the movie: {movie_name}")
recommendations = recommend_movies(movie_name)
print(recommendations)

Recommendation for the movie: Bangkok Breaking
988                              Let's Fight Ghost
8777                                     Yes or No
8029                                    Skin Trade
657                                       The Mole
2194                                  Unlucky Ploy
7017    How to Live Mortgage Free with Sarah Beeny
4840                                    Bad Genius
8778                                   Yes or No 2
3883                                 The Outsiders
5716                                      Paranoid
Name: title, dtype: object
